# Info

This notebook supports the evaluation of the Soil Water Retention Curve, which describes the relationship between moisture and suction head. Furthermore, the notebook allows to investigate the relationship introduced by Mualem and van Genuchten <br>
to describe water retention and hydraulic conductivity in the unsaturated zone for unconsolidated sediments. <br>

In [70]:
import math
import numpy as np
from ipywidgets import *
import matplotlib.pyplot as plt

# given data (retention) - here you can use the approach to fit parameter to measured data

t1=[0.09,0.12,0.15,0.18,0.21,0.24,0.27,0.3,0.33,0.36,0.39,0.42,0.45]
p1=[2230.546345,577.472177,300.4391307,199.8371285,142.8205223,109.6375793,85.19965286,67.18768129,53.82569358,41.8841783,31.92533514,21.62546735,10.23974185]
t2=[0.18,0.19,0.22,0.25,0.28,0.31,0.35,0.4,0.44,0.47,0.51,0.54,0.55]
p2=[50030.534,9000.477,2000.407,900.835,500.023,120.633,60.528,30.189,11.823,7.883,1.514,0.625,0.285]
t3=[0.35,0.37,0.4,0.42,0.44,0.47,0.49,0.5,0.52,0.54,0.55,0.57,0.57]
p3=[350030.55,7800.21,1800.47,940.88,440.03,134.63,56.12,22.11,8.68,4.17,1.94,0.35,0.15]

#definition of the function (conductivity)
def RTC_VG_MUA(tr, ts, alpha, n, plot1, plot2, plot3, plot4):
    x_max = 300
    
    # intermediate results 
    m   = 1-1/n                                         # van Genuchten parameter
    PWP = tr + (ts - tr)/(1+(alpha*10**4.2)**n)**m      # permanent wilting point
    FC  = tr + (ts - tr)/(1+(alpha*10**1.8)**n)**m      # field capacity
    eFC = FC - PWP                                      # effective field capacity

    # model output
    t_plot  = []                                        # t  = theta = moisture content
    p_plot  = []                                        # p  = phi   = suction head
    kr_plot = []                                        # kr = rel. permeability
    
    for x in range (0, x_max):
        t = tr + (ts-tr)*x/(x_max-1)                    # [-] moisture content; please note that range counts up to x_max-1
        te = (t-tr)/(ts-tr)                             # [-] effective saturation      
        if x == 0:
            p     = 1E18                                # [cm] suction head
            kr    = 0                                   # [-] relative hydraulic conductivity
        else: 
            p     = ((te**(-1/m)-1)**(1/n))/alpha                      
            kr    = np.sqrt(te)*(1-(1-te**(1/m))**m)**2
        t_plot.append(t)
        p_plot.append(p)
        kr_plot.append(kr)
        
    
    fig = plt.figure(figsize=(9,6))
    ax  = fig.add_subplot()
    ax.plot(t_plot, p_plot, 'r', markersize=3)
    ax.vlines(x= tr, ymin=1e-1, ymax=1e+5, linestyle='--')      
    ax.hlines(y= 10**4.2, xmin=0, xmax=PWP, colors='g')    #upper green line
    ax.vlines(x= PWP, ymin=1e-1, ymax=10**4.2, colors='g')
    ax.hlines(y= 10**1.8, xmin=0, xmax=FC, colors='b')     #bottom green line
    ax.vlines(x= FC, ymin=1e-1, ymax=10**1.8, colors='b')
    if plot1 == 1:
        ax.plot(t1, p1,'ro', markersize=3)
    if plot2 == 1:
        ax.plot(t2, p2,'bo', markersize=3)
    if plot3 == 1:
        ax.plot(t3, p3,'go', markersize=3)
    ax.set(xlabel='water content [-]', ylabel ='suction head [cm]', xlim = [0, 0.7], ylim = [1e-1,1e+5], yscale = 'log' )
    ax.grid(which="both", color='grey',linewidth=0.5)
    plt.show()

    if plot4 == 1:
        fig = plt.figure(figsize=(6,4))
        ax  = fig.add_subplot()
        ax.plot(t_plot, kr_plot, 'b', markersize = 3)
        ax.set(xlabel='water content [-]', ylabel='rel hydraulic conductivity [cm/d]', xlim = [0, 0.7], ylim = [0,1] )
        ax.grid(which="major", color='grey',linewidth=0.5)
        plt.show()
    
    print ('Van Genuchten             m:', '{:.5f}'.format(m) )
    print ('Permanent Wilting Point PWP:', '{:.2f}'.format(PWP) )
    print ('Field Capacity           FC:', '{:.2f}'.format(FC) )
    print ('Eff. Field Capacity     eFC:', '{:.2f}'.format(eFC) )

The Notebook displays the retention function in red. 
By marking the checkbox "Plot data1/2/3", three different example sets of measured data can be shown within the plot. Try to inversely fit the curve to each data set by manipulating the four parameters with the given sliders (just move your mouse over the data and modify in the textbox or type the values directly). Write down your estimates for the four parameters and try to figure out, which data set belongs to which kind of soil: 
- Medium sand
- Loamy sand
- Clay

Additionally, the relative hydraulic conductivity can be plotted in an extra diagram in blue by marking the check box "Plot $k_r$". Make a screenshot, each time you managed to fit the curve to one of the data sets, and compare the three different graphs, after you fitted the curve to each of the three soils. <br>

Finally, the following soil properties are computed: <br>

van Genuchten parameter m = 1 - 1/n, permanent wilting point (PWP), field capacity (FC), and effective field capacity (eFC). <br>

|symbol | input parameters			    |units	    |remarks
|:------|:------------------------------|:----------|:----------------------
|$\theta_r$|residual moisture content		|	-	    |enter number between 0 and 1 required
|$\theta_s$    |moisture content at saturation	|	-	    |enter number between residual moisture content and 1 required
|$\alpha$|shape parameter a			    |   1/cm	|enter positive number
|$n$    |shape parameter n			    |   -	    |enter number > 1

In [71]:
interact(RTC_VG_MUA,
         tr    = widgets.FloatSlider(value=0.05, min=0.01,    max=0.4, step=0.01, description='$theta_r$ [-]:', disabled=False, continuous_update=False),
         ts    = widgets.FloatSlider(value=0.5 , min=0.2,     max=0.7, step=0.01, description='$theta_s$ [-]:', disabled=False, continuous_update=False),
         alpha = widgets.FloatSlider(value=0.1 , min=0.01,    max=1,   step=0.02, description='$alpha$ [1/cm]:', disabled=False, continuous_update=False),
         n     = widgets.FloatSlider(value=1.2 , min=1.01,    max=3,   step=0.02, description= 'n  [-]:', disabled=False, continuous_update=False),
         plot1 = widgets.Checkbox(value=False, description='Plot data1 ',disabled=False),    
         plot2 = widgets.Checkbox(value=False, description='Plot data2 ',disabled=False),       
         plot3 = widgets.Checkbox(value=False, description='Plot data3 ',disabled=False),       
         plot4 = widgets.Checkbox(value=False, description='Plot $k_r$ ',disabled=False),
#         plot1 = widgets.IntSlider  (value=0   , min=0,       max=1,   step=1,    description='Plot data ',disabled=False),
#         plot2 = widgets.IntSlider  (value=0   , min=0,       max=1,   step=1,    description='Plot $k_r$ ',disabled=False),
         )

interactive(children=(FloatSlider(value=0.05, continuous_update=False, description='$theta_r$ [-]:', max=0.4, …

<function __main__.RTC_VG_MUA(tr, ts, alpha, n, plot1, plot2, plot3, plot4)>